# Variant calling
The next step is performing variant calling to compare the DNA in your samples with that of the reference genome. Maybe some mutation is responsible for the sudden proliferation of these organisms...<br><br>
### 1. Merge files
To perform variant calling you can use the mappings from all your samples. To do that, sort the mappings of each of your samples (tip: you should use `samtools sort` for that). Then, you can merge the sorted files into a single BAM file (tip: you should use `samtools merge` for that). Try to do all these steps so that the header information is present in the final merged BAM file (tip: check the parameters available in the help of the different `samtools` commands)

In [1]:
cd
cd ./Genomics

In [2]:
# First we need to transform all my sam files to (unsorted) bam files 
samtools view -bS hightemp_01.sam > hightemp_01.bam
samtools view -bS hightemp_02.sam > hightemp_02.bam
samtools view -bS normal_01.sam > normal_01.bam
samtools view -bS normal_02.sam > normal_02.bam

# Now we can sort the BAM files
samtools sort hightemp_01.bam > hightemp_01_sorted.bam
samtools sort hightemp_02.bam > hightemp_02_sorted.bam
samtools sort normal_01.bam > normal_01_sorted.bam
samtools sort normal_01.bam > normal_02_sorted.bam

# And now we can merge the files
samtools merge -h hightemp_01.sam -h hightemp_02.sam hightemp.bam hightemp_01_sorted.bam hightemp_02_sorted.bam
samtools merge -h normal_01.sam -h normal_02.sam normal.bam normal_01_sorted.bam normal_02_sorted.bam

In [15]:
ls # check that files were created

hightemp.bam            hightemp_02.sam         normal_01.sam
hightemp_01.bam         hightemp_02_sorted.bam  normal_01_sorted.bam
hightemp_01.sam         mappingStats.txt        normal_02.bam
hightemp_01_sorted.bam  normal.bam              normal_02.sam
hightemp_02.bam         normal_01.bam           normal_02_sorted.bam


<br><br>
### 2. Variant calling
Finally, perform the variant calling using `bcftools mpileup` and `bcftools call`. Check the parameters used in the practice lessons and apply those.  (tip: remember that you can pipe commands in linux using “|”).


In [16]:
# First we need to index the reference genome again, but this time with samtools faidx
cd
samtools faidx ./data/refs/genome.fna

[E::fai_build_core] Different line length in sequence 'NC_000918.1'
[faidx] Could not build fai index ./data/refs/genome.fna.fai


: 1

In [4]:
awk '/^>/ {print;next;} {print length($0);}' ./data/refs/genome.fna | uniq -c

      1 >NC_000918.1 Aquifex aeolicus VF5 chromosome, complete genome
  19391 80
      1 1681


There is a problem with the length of the reads: most lines have a length of 80 and 1 has a length of 1681 characters. In order to build the genome index we need to fix this. It is likely that these lines are the last lines of the sequence, so we can fix the problem by changing the length of those lines manually. <br> <br>

In [22]:
# After edditing, we run the same command
samtools faidx ./Genomics/edited_genome.fna 

# Now we can use bcftools mpileup to transform the BAM file in VCF format
bcftools mpileup -f edited_genome.fna hightemp.bam > hightemp.bcf
bcftools mpileup -f edited_genome.fna normal.bam > normal.bcf

# Finally, we  call the SNPs using bcftools call
bcftools call -vc hightemp.bcf > final_hightemp.bcf 
bcftools call -vc normal.bcf > final_normal.bcf 

<br><br>
### 3. Transform file
Now we can look into the files created to answer the questions
Transform, to a “tab” separated file, the BCF file you obtained, including the variant ID, its position, the reference allele, the alternative allele, the variant quality and the depth of coverage of the variant (tip: use `bcftools view calls.bcf | grep -v "^#" | cut -f 1,2,4,5,6,8 | sed 's#DP=\([0-9][0-9]*\).*#\1#' > calls.tsv`. You could run the previous command line with and without the `sed` command to check what is its purpose).


In [39]:
# overview of how the file looks
bcftools view final_hightemp.bcf

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##bcftoolsVersion=1.8+htslib-1.8
##bcftoolsCommand=mpileup -f edited_genome.fna hightemp.bam
##reference=file://edited_genome.fna
##contig=<ID=NC_000918.1,length=1552961>
##ALT=<ID=*,Description="Represents allele(s) other than observed.">
##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">
##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">
##INFO=<ID=RPB,Number=1,Type=Float,Description="Mann-Whitney U test of Read Position Bias (bigger is better)">
##INFO=<ID=MQB,Number=1,Type=Float,Description="Mann-Whitney U test of M

NC_000918.1	420897	.	T	C	3.5427	.	DP=3;SGB=-0.379885;RPB=1;MQB=1;BQB=1;MQ0F=0;AF1=0.499801;AC1=1;DP4=0,2,0,1;MQ=42;FQ=5.4644;PV4=1,1,1,1	GT:PL	0/1:31,0,64
NC_000918.1	453861	.	A	G	3.5427	.	DP=3;SGB=-0.379885;RPB=1;MQB=1;BQB=1;MQ0F=0;AF1=0.499801;AC1=1;DP4=0,2,0,1;MQ=41;FQ=5.4644;PV4=1,1,0,1	GT:PL	0/1:31,0,64
NC_000918.1	469402	.	A	G	5.46076	.	DP=2;SGB=-0.379885;RPB=1;MQB=1;BQB=1;MQ0F=0;AF1=0.5;AC1=1;DP4=0,1,0,1;MQ=42;FQ=5.45858;PV4=1,1,1,1	GT:PL	0/1:34,0,34
NC_000918.1	521097	.	T	A	3.5427	.	DP=3;SGB=-0.379885;RPB=1;MQB=1;BQB=1;MQ0F=0;AF1=0.499801;AC1=1;DP4=0,2,0,1;MQ=41;FQ=5.4644;PV4=1,1,1,0	GT:PL	0/1:31,0,64
NC_000918.1	521108	.	G	T	3.5427	.	DP=3;SGB=-0.379885;RPB=1;MQB=1;BQB=1;MQ0F=0;AF1=0.499801;AC1=1;DP4=0,2,0,1;MQ=41;FQ=5.4644;PV4=1,1,0,1	GT:PL	0/1:31,0,64
NC_000918.1	531346	.	T	C	4.76875	.	DP=14;VDB=0.22;SGB=-0.453602;RPB=0.583333;MQB=0.5;BQB=0.916667;MQ0F=0;AF1=0.499875;AC1=1;DP4=0,12,0,2;MQ=42;FQ=6.98503;PV4=1,1,0.00376607,1	GT:PL	0/1:33,0,170
NC_000918.1	544392	.	T	C	3.5427	.	

NC_000918.1	1155695	.	T	A	14.176	.	DP=10;VDB=0.02;SGB=-0.453602;RPB=0.9375;MQB=0.75;BQB=0.875;MQ0F=0;AF1=0.49999;AC1=1;DP4=8,0,2,0;MQ=41;FQ=17.1004;PV4=1,1,0.272369,1	GT:PL	0/1:44,0,154
NC_000918.1	1158878	.	A	C	3.5427	.	DP=3;SGB=-0.379885;RPB=1;MQB=1;BQB=1;MQ0F=0;AF1=0.499801;AC1=1;DP4=2,0,1,0;MQ=41;FQ=5.4644;PV4=1,1,0,1	GT:PL	0/1:31,0,64
NC_000918.1	1168946	.	G	A	3.5427	.	DP=3;SGB=-0.379885;RPB=1;MQB=1;BQB=1;MQ0F=0;AF1=0.499802;AC1=1;DP4=0,2,0,1;MQ=42;FQ=5.46285;PV4=1,1,1,0.383046	GT:PL	0/1:31,0,61
NC_000918.1	1171193	.	A	T	3.54318	.	DP=2;SGB=-0.379885;RPB=1;MQB=1;BQB=1;MQ0F=0;AF1=0.499901;AC1=1;DP4=1,0,1,0;MQ=42;FQ=4.27965;PV4=1,1,1,1	GT:PL	0/1:31,0,34
NC_000918.1	1183460	.	T	A	4.76933	.	DP=2;SGB=-0.379885;RPB=1;MQB=1;BQB=1;MQ0F=0;AF1=0.499974;AC1=1;DP4=1,0,1,0;MQ=42;FQ=5.08697;PV4=1,1,1,1	GT:PL	0/1:33,0,34
NC_000918.1	1190872	.	A	C	3.54285	.	DP=3;SGB=-0.379885;RPB=1;MQB=1;BQB=1;MQ0F=0;AF1=0.499833;AC1=1;DP4=0,2,0,1;MQ=41;FQ=5.01828;PV4=1,1,0,1	GT:PL	0/1:31,0,39
NC_000918.1	1190877	

In [3]:
cd 
cd ./Genomics
ls

edited_genome.fna      hightemp_01.bam         normal.bcf
edited_genome.fna.fai  hightemp_01.bcf         normal_01.bam
final_hightemp.bcf     hightemp_01.sam         normal_01.bcf
final_hightemp_01.bcf  hightemp_01_sorted.bam  normal_01.sam
final_hightemp_02.bcf  hightemp_02.bam         normal_01_sorted.bam
final_normal.bcf       hightemp_02.bcf         normal_02.bam
final_normal_01.bcf    hightemp_02.sam         normal_02.bcf
final_normal_02.bcf    hightemp_02_sorted.bam  normal_02.sam
hightemp.bam           mappingStats.txt        normal_02_sorted.bam
hightemp.bcf           normal.bam


In [4]:
bcftools view hightemp.bcf | grep -v "^#" | cut -f 1,2,4,5,6,8 | sed 's#DP=\([0-9][0-9]*\).*#\1#' > hightemp.tsv
bcftools view normal.bcf | grep -v "^#" | cut -f 1,2,4,5,6,8 | sed 's#DP=\([0-9][0-9]*\).*#\1#' > normal.tsv

<br><br>
### 4. Analyse results
<br>The first thing we can see here is that we need to remove the header when we work with the files in order to analyse the data using commands (we can access the data by columns). We can do so by using the command `tail -n+33`.

__How many variants did you obtain? How many are SNPs, how many insertions and how many deletions?__

In [55]:
# number of uniq variants
tail -n+33 final_hightemp.bcf | cut -f4-5 | sort | uniq -c| wc -l
tail -n+33 final_normal.bcf | cut -f4-5 | sort | uniq -c| wc -l

12
12


In [59]:
bcftools stats final_hightemp.bcf # gives info about the total number of variants, SNPs and indels

# This file was produced by bcftools stats (1.8+htslib-1.8) and can be plotted using plot-vcfstats.
# The command line was:	bcftools stats  final_hightemp.bcf
#
# Definition of sets:
# ID	[2]id	[3]tab-separated file names
ID	0	final_hightemp.bcf
# SN, Summary numbers:
#   number of records   .. number of data rows in the VCF
#   number of no-ALTs   .. reference-only sites, ALT is either "." or identical to REF
#   number of SNPs      .. number of rows with a SNP
#   number of MNPs      .. number of rows with a MNP, such as CC>TT
#   number of indels    .. number of rows with an indel
#   number of others    .. number of rows with other type, for example a symbolic allele or
#                          a complex substitution, such as ACT>TCGA
#   number of multiallelic sites     .. number of rows with multiple alternate alleles
#   number of multiallelic SNP sites .. number of rows with multiple alternate alleles, all SNPs
# 
#   Note that rows containing multiple types will be counted m

In [60]:
bcftools stats final_normal.bcf # gives info about the total number of variants, SNPs and indels

# This file was produced by bcftools stats (1.8+htslib-1.8) and can be plotted using plot-vcfstats.
# The command line was:	bcftools stats  final_normal.bcf
#
# Definition of sets:
# ID	[2]id	[3]tab-separated file names
ID	0	final_normal.bcf
# SN, Summary numbers:
#   number of records   .. number of data rows in the VCF
#   number of no-ALTs   .. reference-only sites, ALT is either "." or identical to REF
#   number of SNPs      .. number of rows with a SNP
#   number of MNPs      .. number of rows with a MNP, such as CC>TT
#   number of indels    .. number of rows with an indel
#   number of others    .. number of rows with other type, for example a symbolic allele or
#                          a complex substitution, such as ACT>TCGA
#   number of multiallelic sites     .. number of rows with multiple alternate alleles
#   number of multiallelic SNP sites .. number of rows with multiple alternate alleles, all SNPs
# 
#   Note that rows containing multiple types will be counted multi

<br><br>
__How many variants have quality greater or equal than 10?__

In [6]:
tail -n+33 final_hightemp.bcf | cut -f6 | awk '{if($1>=10)print $1}' | wc -l
tail -n+33 final_normal.bcf | cut -f6 | awk '{if($1>=10)print $1}' | wc -l

11
984


<br><br>
__How many variants have depth of coverage greater or equal than 10?__

In [16]:
tail -n+33 final_hightemp.bcf | cut -f8 |tr ';' "\t"|cut -f 1 | sed -e 's/DP=//g' | awk '$1>10 {print;} '| wc -l
tail -n+33 final_normal.bcf | cut -f8 |tr ';' "\t"|cut -f 1 | sed -e 's/DP=//g' | awk '$1>10 {print;} '| wc -l

15
1141


<br><br>
__Identify the variant with best quality. Could this variant be affecting a gene? (tip: compare the position of the variant with the positions of the genes in the genome GFF file). Which gene did you find, if any? Without actually checking it, could you give an example of how your variant could be affecting a gene product (e.g. a protein)?__

In [6]:
# find variant with best quality
tail -n+33 final_hightemp.bcf | sort -rn -k6 | head -n 1
echo ''
tail -n+33 final_normal.bcf | sort -rn -k6 | head -n 1

NC_000918.1	135330	.	A	C	221.999	.	DP=84;VDB=0.222872;SGB=-0.693147;MQSB=0.800287;MQ0F=0;AF1=1;AC1=2;DP4=0,0,33,45;MQ=41;FQ=-261.989	GT:PL	1/1:255,235,0

NC_000918.1	135330	.	A	C	221.999	.	DP=248;VDB=0.978942;SGB=-0.693147;MQSB=0.954788;MQ0F=0;AF1=1;AC1=2;DP4=0,0,127,101;MQ=41;FQ=-281.989	GT:PL	1/1:255,255,0
sort: write failed: standard output: Broken pipe
sort: write error


In [11]:
# find the gene with that position in the gff file
cd
cd ./data/refs
awk -F"\t" '$4<=135330' genes.gff | awk -F"\t" '$5>=135330'

NC_000918.1	RefSeq	gene	134585	136105	.	-	0	ID=cds158;Parent=gene163;Dbxref=Genbank:NP_213151.1,GeneID:1192825;Name=NP_Unk02;gbkey=CDS;gene=nifA;product=NifA


<br>The result is the same for both samples, a A ➝ C at position 135330. Since this mutation is just of one position, if the mutation were to be in the coding part of the gene, only one aminoacid would be changing (that we know of, probably we should check other SNPs in the sequence of this gene). This is only a small change, but it could have a big impact in the tertiary or quaternary structure of the coding protein and affect its performance significantly. Additionally, if the mutation was not in the coding region of the gene, it could affect the promoter union region, or an inhibitor union region and therefore affect the transcription of the gene (increasing or decreasing it). It could also be a mutation that affected the translation of the protein by changing the ribosome union or its detachment from the mRNA.<br>
About the gene affected, it is `NifA` which (accordint to UniProt) corresponds to a nitrogen fixation specific regulatory protein. Given that we are studying  a bacteria that blooms in high temperatures, it is posible that this mutation is increasing the fixation rate or making the process more efficient to make up for the disappearance of other nitrogen fixation organisms due to the temperature raise. 

<br><br>
__Repeat the variant calling using only a single .bam file, instead of merging them. What is the main difference you find in the results when you use only one file?__
<br>The main difference is that the number of reads for most of the characteristics we filtered before does not sum up to what the merged files had. For the high temperature samples, the sum of the two individual BAMs is greatly over the count for the merged file; on the other hand, the sum of the individual BAMs for the normal sample is significantly smaller than in the merged file. 

In [30]:
bcftools mpileup -f edited_genome.fna hightemp_01_sorted.bam > hightemp_01.bcf
bcftools mpileup -f edited_genome.fna hightemp_02_sorted.bam > hightemp_02.bcf
bcftools mpileup -f edited_genome.fna normal_01_sorted.bam > normal_01.bcf
bcftools mpileup -f edited_genome.fna normal_02_sorted.bam > normal_02.bcf

bcftools call -vc hightemp_01.bcf > final_hightemp_01.bcf 
bcftools call -vc hightemp_02.bcf > final_hightemp_02.bcf 
bcftools call -vc normal_01.bcf > final_normal_01.bcf 
bcftools call -vc normal_02.bcf > final_normal_02.bcf 

Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid


In [3]:
# number of variants
tail -n+33 final_hightemp_01.bcf | cut -f4-5 | sort | uniq -c| wc -l
tail -n+33 final_hightemp_02.bcf | cut -f4-5 | sort | uniq -c| wc -l
tail -n+33 final_normal_01.bcf | cut -f4-5 | sort | uniq -c| wc -l
tail -n+33 final_normal_02.bcf | cut -f4-5 | sort | uniq -c| wc -l

12
12
12
12


In [4]:
# number of SNPs
tail -n+33 final_hightemp_01.bcf | wc -l
tail -n+33 final_hightemp_02.bcf | wc -l
tail -n+33 final_normal_01.bcf | wc -l
tail -n+33 final_normal_02.bcf | wc -l

397
418
373
373


In [5]:
# quality check
tail -n+33 final_hightemp_01.bcf |cut -f6 | awk '{if($1>=10)print $1}' | wc -l
tail -n+33 final_hightemp_02.bcf | cut -f6 | awk '{if($1>=10)print $1}' | wc -l
tail -n+33 final_normal_01.bcf | cut -f6 | awk '{if($1>=10)print $1}' | wc -l
tail -n+33 final_normal_02.bcf | cut -f6 | awk '{if($1>=10)print $1}' | wc -l

51
76
82
82


In [6]:
tail -n+33 final_hightemp_01.bcf | cut -f8 |tr ';' "\t"|cut -f 1 | sed -e 's/DP=//g' | awk '$1>10 {print;} '| wc -l
tail -n+33 final_hightemp_02.bcf | cut -f8 |tr ';' "\t"|cut -f 1 | sed -e 's/DP=//g' | awk '$1>10 {print;} '| wc -l
tail -n+33 final_normal_01.bcf | cut -f8 |tr ';' "\t"|cut -f 1 | sed -e 's/DP=//g' | awk '$1>10 {print;} '| wc -l
tail -n+33 final_normal_02.bcf | cut -f8 |tr ';' "\t"|cut -f 1 | sed -e 's/DP=//g' | awk '$1>10 {print;} '| wc -l

65
73
67
67


In [7]:
tail -n+33 final_hightemp_01.bcf | sort -rn -k6 | head -n 1

NC_000918.1	135330	.	A	C	221.999	.	DP=51;VDB=0.345422;SGB=-0.693147;MQSB=0.50373;MQ0F=0;AF1=1;AC1=2;DP4=0,0,20,28;MQ=41;FQ=-170.988	GT:PL	1/1:255,144,0


In [8]:
tail -n+33 final_hightemp_02.bcf | sort -rn -k6 | head -n 1

NC_000918.1	135330	.	A	C	221.999	.	DP=33;VDB=0.299086;SGB=-0.693097;MQSB=0.916147;MQ0F=0;AF1=1;AC1=2;DP4=0,0,13,17;MQ=41;FQ=-116.987	GT:PL	1/1:255,90,0
sort: fflush failed: standard output: Broken pipe
sort: write error


In [9]:
tail -n+33 final_normal_01.bcf | sort -rn -k6 | head -n 1

NC_000918.1	135330	.	A	C	221.999	.	DP=218;VDB=0.65036;SGB=-0.693147;RPB=1;MQB=1;MQSB=0.933189;BQB=1;MQ0F=0;AF1=1;AC1=2;DP4=0,1,93,114;MQ=41;FQ=-281.989;PV4=1,0.411684,1,1	GT:PL	1/1:255,255,0


In [10]:
tail -n+33 final_normal_02.bcf | sort -rn -k6 | head -n 1

NC_000918.1	135330	.	A	C	221.999	.	DP=218;VDB=0.65036;SGB=-0.693147;RPB=1;MQB=1;MQSB=0.933189;BQB=1;MQ0F=0;AF1=1;AC1=2;DP4=0,1,93,114;MQ=41;FQ=-281.989;PV4=1,0.411684,1,1	GT:PL	1/1:255,255,0


<br><br>
__(extra question which you may take some time...) Download, to your local machine, the files with the mappings and the fasta file of the genome. Use them to locate in IGV the best variant you have, and capture the image of the variant. Note that you will likely need the indexes of the mappings (.bai) and genome (.fai) files.__

In [15]:
# I need to create an index of the sorted bam files in order to visualize them
samtools index hightemp_01_sorted.bam
samtools index hightemp_02_sorted.bam
samtools index normal_01_sorted.bam
samtools index normal_02_sorted.bam
ls | grep 'bai'

hightemp_01_sorted.bam.bai
hightemp_02_sorted.bam.bai
normal_01_sorted.bam.bai
normal_02_sorted.bam.bai


### IGV result

![alt text](./IGV.png "IGV result")